# Time Series

- https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/


In [2]:
import pandas as pd

## Time series data structures

In [ ]:
# to do

## Creating a time series DataFrame

In [ ]:
# Open Power Systems Data
opsd_daily = pd.read_csv('opsd_germany_daily.csv')